In [13]:
import cv2
import numpy as np
import os
from PIL import Image

In [14]:
images=os.listdir('Images_of_faces')
crop_images=os.listdir('Cropes_of_faces')
for image in images:
    for crop_image in crop_images:
        if image==crop_image:
            original_image = Image.open('Images_of_faces/'+image)
            cropped_image=Image.open('Cropes_of_faces/'+image)
            original_image = np.array(original_image)
            cropped_image = np.array(cropped_image)
            height, width, _= cropped_image.shape
            h, w, _=original_image.shape
            result = cv2.matchTemplate(original_image, cropped_image, cv2.TM_CCOEFF_NORMED)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
            top_left = max_loc
            bottom_right = (top_left[0] + width, top_left[1] + height)
            with open('boxes2/'+image, mode='w') as txt_file:
                txt_file.write(str(top_left[0]/w)+' '+str(top_left[1]/h)+' '+str(bottom_right[0]/w)+' '+str(bottom_right[1]/h))

In [15]:
def calculate_iou(box1, box2):
    """
    box1 и box2: [x1, y1, x2, y2], где (x1, y1) — левый верхний угол, (x2, y2) — правый нижний.
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area_box1 + area_box2 - intersection
    iou = intersection / union if union != 0 else 0
    return iou

In [16]:
def calculate_precision_recall(gt_boxes, pred_boxes, iou_threshold=0.5):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for pred_box in pred_boxes:
        matched = False
        for gt_box in gt_boxes:
            iou = calculate_iou(gt_box, pred_box)
            if iou > iou_threshold:
                true_positives += 1
                matched = True
                break
        if not matched:
            false_positives += 1
            
    false_negatives = len(gt_boxes) - true_positives
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    return precision, recall

In [17]:
import numpy as np

def compute_ap(recalls, precisions):
    """Вычисляет Average Precision по кривой precision-recall"""
    # Добавляем граничные точки
    recalls = np.concatenate(([0.], recalls, [1.]))
    precisions = np.concatenate(([0.], precisions, [0.]))
    
    # Сглаживаем кривую precision
    for i in range(len(precisions) - 1, 0, -1):
        precisions[i - 1] = np.maximum(precisions[i - 1], precisions[i])
    
    # Находим точки, где recall меняется
    i = np.where(recalls[1:] != recalls[:-1])[0]
    
    # Вычисляем площадь под кривой
    ap = np.sum((recalls[i + 1] - recalls[i]) * precisions[i + 1])
    return ap

def calculate_map50_simple(pred_boxes, true_boxes, iou_threshold=0.5):
    """
    Вычисляет mAP50 для одного класса с простыми боксами
    
    Параметры:
    pred_boxes (list): Список предсказанных боксов в формате [[x1,y1,x2,y2], ...]
    true_boxes (list): Список истинных боксов в формате [[x1,y1,x2,y2], ...]
    iou_threshold (float): Порог IoU для совпадения (по умолчанию 0.5)
    
    Возвращает:
    float: Значение mAP50
    """
    # Для простоты считаем, что все предсказания имеют confidence=1.0
    # и все принадлежат одному "изображению"
    
    # Количество истинных объектов
    num_gt = len(true_boxes)
    
    if num_gt == 0:
        return 0.0
    
    # Инициализация массивов для TP и FP
    tp = np.zeros(len(pred_boxes))
    fp = np.zeros(len(pred_boxes))
    
    # Копируем true_boxes для отметки сопоставленных
    remaining_gt_boxes = [box for box in true_boxes]
    
    # Для каждого предсказания (у нас нет confidence, считаем все равнозначными)
    for i, pred_box in enumerate(pred_boxes):
        best_iou = 0
        best_gt_idx = -1
        
        # Ищем наилучшее совпадение с истинными боксами
        for j, gt_box in enumerate(remaining_gt_boxes):
            if gt_box is None:
                continue
                
            iou = calculate_iou(pred_box, gt_box)
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = j
        
        # Определяем true positive или false positive
        if best_iou >= iou_threshold and best_gt_idx != -1:
            tp[i] = 1
            remaining_gt_boxes[best_gt_idx] = None  # Помечаем как сопоставленный
        else:
            fp[i] = 1
    
    # Вычисляем кумулятивные суммы TP и FP
    tp_cumsum = np.cumsum(tp)
    fp_cumsum = np.cumsum(fp)
    
    # Вычисляем precision и recall
    recalls = tp_cumsum / num_gt
    precisions = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-6)
    
    # Вычисляем AP (Average Precision)
    ap = compute_ap(recalls, precisions)
    
    return ap

In [18]:
gt_boxes=[]
pred_boxes=[]
labels=os.listdir('my_cropes/obj_Train_data')
boxes=os.listdir('boxes2')
for label in labels:
    for box in boxes:
        if label[:24]==box[:24]:
            with open('boxes2/'+box) as box_file:
                content1 = box_file.read()
            with open('my_cropes/obj_Train_data/'+label) as label_file:
                content2=label_file.read()
            pred_box=content1.split('\n')[0]
            pred_box = [float(item) for item in pred_box.split()]
            pred_boxes.append(pred_box)
            for gt_box in content2.split('\n'):
                gt_box=gt_box.split()[1:]
                if len(gt_box)==4:
                    gt_box = [float(gt_box[0])-float(gt_box[2])/2, float(gt_box[1])-float(gt_box[3])/2, float(gt_box[0])+float(gt_box[2])/2, float(gt_box[1])+float(gt_box[3])/2]
                    gt_boxes.append(gt_box)
            print(box)
                
precision, recall = calculate_precision_recall(gt_boxes, pred_boxes)
print(precision, recall)

2025-04-01-16-16-17_Холл_ВКИ_(Активный)_Определение лиц.jpg
2025-04-01-16-16-37_Холл_ВКИ_(Активный)_Определение лиц.jpg
2025-04-01-16-16-51_Холл_ВКИ_(Активный)_Определение лиц.jpg
2025-04-01-16-17-03_Коридор_1_этаж_ВКИ_(1)_Определение лиц.jpg
2025-04-01-16-17-04_Коридор_1_этаж_ВКИ_(1)_Определение лиц.jpg
2025-04-01-16-17-05_Коридор_1_этаж_ВКИ_(1)_Определение лиц.jpg
2025-04-01-16-17-08_Коридор_1_этаж_ВКИ_(1)_Определение лиц (1).jpg
2025-04-01-16-17-08_Коридор_1_этаж_ВКИ_(1)_Определение лиц.jpg
2025-04-01-16-17-08_Коридор_1_этаж_ВКИ_(1)_Определение лиц (1).jpg
2025-04-01-16-17-08_Коридор_1_этаж_ВКИ_(1)_Определение лиц.jpg
2025-04-01-16-17-10_Коридор_1_этаж_ВКИ_(1)_Определение лиц.jpg
2025-04-01-16-17-12_Коридор_1_этаж_ВКИ_(1)_Определение лиц.jpg
2025-04-01-16-17-15_Коридор_1_этаж_ВКИ_(1)_Определение лиц.jpg
2025-04-01-16-17-17_Коридор_1_этаж_ВКИ_(1)_Определение лиц.jpg
2025-04-01-16-17-19_Коридор_1_этаж_ВКИ_(1)_Определение лиц.jpg
2025-04-01-16-17-22_Коридор_1_этаж_ВКИ_(1)_Определение л

In [19]:
print(calculate_map50_simple(pred_boxes, gt_boxes))

0.32240074357236836


In [ ]:
image='2025-03-22-14-54-39_TEST_ЛЮДИ_АВТОБУС_Определение лиц.jpg'
original_image = Image.open('Images_of_faces/'+image)
original_image.show()

In [ ]:
cropped_image = Image.open('Cropes_of_faces/'+image)
cropped_image.show()

In [11]:
import cv2
import PIL
import mediapipe as mp
import numpy as np

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

def get_face_roll_mediapipe(image):
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    if not results.multi_face_landmarks:
        return None
    
    # Берем первую обнаруженную лицо
    landmarks = results.multi_face_landmarks[0]
    
    # Нос (вершина) и подбородок
    nose_tip = landmarks.landmark[4]  # Примерная точка кончика носа
    chin = landmarks.landmark[152]    # Примерная точка подбородка
    
    # Вычисляем вектор "нос-подбородок" (ось Y лица)
    dy = chin.y - nose_tip.y
    dx = chin.x - nose_tip.x
    roll = np.degrees(np.arctan2(dy, dx)) - 90  # Корректировка на вертикаль
    
    return roll


image = PIL.Image.open("C:/Users/tsire/Downloads/Cropes_of_faces/2025-04-01-16-25-52_Холл_ВКИ_(Активный)_Определение лиц.jpg")
image=np.array(image)
roll_angle = get_face_roll_mediapipe(image)
print(f"Roll angle (MediaPipe): {roll_angle:.2f}°")

Roll angle (MediaPipe): 21.83°


In [14]:
import mediapipe as mp
import numpy as np

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

def get_face_pitch_mediapipe(image):
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    if not results.multi_face_landmarks:
        return None
    
    landmarks = results.multi_face_landmarks[0]
    
    # Нос (1), подбородок (152), лоб (10)
    nose = landmarks.landmark[1]
    chin = landmarks.landmark[152]
    forehead = landmarks.landmark[10]
    
    # Вектор "нос-лоб" (ось Z)
    dz = forehead.z - nose.z
    dy = forehead.y - nose.y
    pitch = np.degrees(np.arctan2(dy, dz))
    
    return pitch

image = PIL.Image.open("C:/Users/tsire/Downloads/2025-04-01-16-25-52_Холл_ВКИ_(Активный)_Определение лиц_upscayl_4x_upscayl-standard-4x.png")
image=np.array(image)
pitch = get_face_pitch_mediapipe(image)
print(f"Pitch (MediaPipe): {pitch:.2f}°")

Pitch (MediaPipe): -92.14°


In [24]:
import mediapipe as mp
import numpy as np

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

def get_face_yaw_mediapipe(image):
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    if not results.multi_face_landmarks:
        return None
    
    landmarks = results.multi_face_landmarks[0]
    
    # Нос (1), левая (454) и правая (234) точки лица
    nose = landmarks.landmark[1]
    left_face = landmarks.landmark[454]
    right_face = landmarks.landmark[234]
    
    # Вектор "нос-левая_точка" и "нос-правая_точка"
    dx_left = nose.x - left_face.x
    dx_right = right_face.x - nose.x
    
    # Расчет угла (нормализованный)
    yaw = np.degrees(math.atan2(dx_right - dx_left, (dx_left + dx_right)))
    
    return yaw

path="C:/Users/tsire/Downloads/Cropes_of_faces/2025-04-01-16-25-52_Холл_ВКИ_(Активный)_Определение лиц.jpg"
image = PIL.Image.open(path)
image=np.array(image)
yaw = get_face_yaw_mediapipe(image)
print(f"Yaw (MediaPipe): {yaw:.2f}°")

Yaw (MediaPipe): -151.29°
